In [ ]:
import cv2
import os
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
os.chdir("../")

In [ ]:
from common.dataset import FolderDataset
from common.draw_utils import show_image, show_images
from common.utils import resize_shortest

In [ ]:
SMART_CV_DOC_LOC_PATH = "/Users/19839701/Developer/datasets/localization/smartcv_doc_loc"

In [ ]:
smartcv_doc_loc = FolderDataset(SMART_CV_DOC_LOC_PATH)

In [ ]:
image = smartcv_doc_loc[0]

# Resize

In [ ]:
new_width = 300
ratio, resized_image = resize_shortest(image, new_width, cv2.INTER_LINEAR)

# Smoothing

In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
smoothed_image = cv2.bilateralFilter(gray_image, 9, 75, 75)

show_images([gray_image, smoothed_image], size=10)

# Channels

In [ ]:
blue_channel, green_channel, red_channel = cv2.split(resized_image)
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
inverted = 255 - gray_image

show_images([blue_channel, green_channel, red_channel, gray_image, inverted], size=10)

# Contrast

In [ ]:
from common.preprocess import make_contrast, apply_brightness_contrast, make_contrast_lab

In [ ]:
contrast_image = make_contrast_lab(resized_image)
show_images([contrast_image, resized_image], size=10)

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

contrast_image = make_contrast(gray_image, alpha = 2, beta = 0)
contrast_image_2 = apply_brightness_contrast(gray_image, brightness = 0, contrast = 20)

contrast_image_3 = cv2.equalizeHist(gray_image)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
contrast_image_4 = clahe.apply(gray_image)

show_images([contrast_image_3, contrast_image_4, gray_image], size=10)

# Sharpen 

In [ ]:
from common.preprocess import unsharp_mask

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)

sharpen_image = np.uint8(1.5 * gray_image - 0.5 * blur_image)

sharpen_image_2 = unsharp_mask(gray_image, threshold=0)

show_images([gray_image, sharpen_image_2, sharpen_image], size=10)

# Canny

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)

lower_sigma=0.3
upper_sigma=1.5
image_median = np.median(blur_image)
lower_thresh = int(max(0, lower_sigma * image_median))
upper_thresh = int(min(255, upper_sigma * image_median))
print(f"image_median: {image_median}, tresh: {lower_thresh} - {upper_thresh}")

edge_image = cv2.Canny(blur_image, lower_thresh, upper_thresh)

show_images([resized_image, blur_image, edge_image], size=9)

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)
edge_image = cv2.Canny(blur_image, 75, 200)

show_images([resized_image, blur_image, edge_image], size=9)

In [ ]:
blur_ks = 5
blur_edge_image = cv2.GaussianBlur(edge_image, (blur_ks, blur_ks), 0)

show_image(blur_edge_image, size=10)

# Connected Components

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)
edge_image = cv2.Canny(blur_image, 75, 200)
show_image(edge_image)

In [ ]:
num_labels, labels_image, stats, centroids = cv2.connectedComponentsWithStats(edge_image)
num_labels

In [ ]:
areas = stats[:, cv2.CC_STAT_AREA]

In [ ]:
AREA_THREASH = 200
filtered_image = np.zeros_like(labels_image)

for label in range(1, num_labels): # from 1 to skip background
    if areas[label] > AREA_THREASH:
        filtered_image[labels_image==label] = 1
        
show_image(filtered_image, size=10)

In [ ]:
labels_good_area = np.arange(1, num_labels)[areas[1:] > AREA_THREASH]
filtered_image = np.zeros_like(labels_image)

for label in labels_good_area:
    filtered_image[labels_image==label] = 1

show_image(filtered_image, size=10)

In [ ]:
comp_image_sizes = []

for label in range(1, num_labels): # from 1 to skip background
    comp_image = np.zeros_like(labels_image)
    comp_image[labels_image==label] = 1
    comp_image_size = np.sum(comp_image)
    comp_image_sizes.append(comp_image_size)
    
comp_image_sizes = np.array(comp_image_sizes)

In [ ]:
comp_filtered = np.arange(1, num_labels)[comp_image_sizes > 100]
filtered_image = np.zeros_like(labels_image)

for comp in comp_filtered:
    filtered_image[labels_image==comp] = 1

show_image(filtered_image, size=10)

In [ ]:
arg_max = sorted(range(len(comp_image_sizes)), key=lambda k: comp_image_sizes[k], reverse=True)
arg_max = np.array(arg_max) + 1

In [ ]:
areas = stats[:, cv2.CC_STAT_AREA]

width = stats[:, cv2.CC_STAT_WIDTH]
height = stats[:, cv2.CC_STAT_HEIGHT]

areas_wh = width * height
iou = areas / areas_wh

In [ ]:
arg_max = sorted(range(len(iou)), key=lambda k: iou[k], reverse=True)
arg_max = np.array(arg_max)

In [ ]:
index = 20
comp_image = np.zeros_like(labels_image)
comp_image[labels_image==arg_max[index]] = 1
print(iou[arg_max[index]], areas[arg_max[index]])

show_image(comp_image, size=10)

# Binarization

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 7
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)

thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 10)

show_images([gray_image, thresh], size=8)

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

show_images([gray_image, thresh], size=8)

# Morphology

In [ ]:
# kernel = np.ones((1,5), np.uint8)
kernel = np.ones((5,1), np.uint8)

# edge_image gray_image
opening = cv2.morphologyEx(edge_image, cv2.MORPH_OPEN, kernel)

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
closing_image = cv2.morphologyEx(edge_image, cv2.MORPH_CLOSE, kernel)

show_image(closing_image, size=7)

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))
dilation_image = cv2.dilate(closing_image, kernel, iterations=1)

show_image(dilation_image, size=7)

# Sobel

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)

# Sobel
ddepth = cv2.CV_64F # CV_64F CV_16S
ksize = 1

sobel_x = cv2.Sobel(blur_image, ddepth, 1, 0, ksize=ksize)
sobel_y = cv2.Sobel(blur_image, ddepth, 0, 1, ksize=ksize)

# Show
show_images([resized_image, sobel_x, sobel_y], size=8)

In [ ]:
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
blur_ks = 5
blur_image = cv2.GaussianBlur(gray_image, (blur_ks, blur_ks), 0)

laplacian = cv2.Laplacian(blur_image, cv2.CV_64F)

# Show
show_images([resized_image, laplacian], size=8)

In [ ]:
sobel_y_blur = cv2.GaussianBlur(sobel_y, (5, 1), 0)